In [1]:
!pip install transformers datasets scikit-learn accelerate

In [ ]:
import pandas as pd

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DLThon/data/submission.csv')
test = pd.read_csv('/content/drive/MyDrive/DLThon/data/test.csv')
train = pd.read_csv('/content/drive/MyDrive/DLThon/data/train.csv')
normal_conv = pd.read_csv('/content/drive/MyDrive/DLThon/normalconversation/normal_conversation.csv')

In [ ]:
submission.head()

,idx,class
0,t_000,NaN
1,t_001,NaN
2,t_002,NaN
3,t_003,NaN
4,t_004,NaN


In [ ]:
test.head()

,idx,conversation
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...


In [ ]:
train.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [ ]:
normal_conv.head()

,idx,class,conversation
0,0,일반 대화,"현우야, 오늘 저녁에 혹시 시간 돼? 맛있는 거 먹으러 갈까?\n오늘? 갑자기 웬일..."
1,1,일반 대화,"혜진야, 이번에 새로 개봉한 액션 영화 봤어? 재밌다던데.\n아니 아직 못 봤어. ..."
2,2,일반 대화,"준호야, 미안한데 혹시 지금 여유 자금 좀 있어?\n갑자기 돈은 왜? 무슨 급한 일..."
3,3,일반 대화,"야 민수, 오늘 수업 끝나고 PC방 가서 게임 한판 할래?\n오, 게임? 요즘 랭크..."
4,4,일반 대화,"준호야, 혹시 어제 수학 숙제 다 했어? 문제 너무 어렵더라.\n응, 나도 겨우 다..."


In [6]:
import pandas as pd
import numpy as np
import torch
import random
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# 1. 설정 및 시드 고정
SEED = 42
MODEL_NAME = "klue/roberta-base"
MAX_LEN = 128  # 메모리 부족 시 128, 여유 있으면 256
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ==========================================
# 2. 데이터 로드 및 전처리 (안전 버전)
# ==========================================
print("Loading data...")
train_df = pd.read_csv('/content/drive/MyDrive/DLThon/data/train.csv')
normal_df = pd.read_csv('/content/drive/MyDrive/DLThon/normal_conv/normal_conversation.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DLThon/data/test.csv')

# 데이터 병합
combined_df = pd.concat([train_df, normal_df], ignore_index=True)

# [중요] 전처리 함수
def custom_preprocessing(text):
    if not isinstance(text, str): return ""
    text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ]){3,}', r'\1\1', text)
    text = re.sub(r'([!?.]{2,})', r'\1', text)
    text = re.sub(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 !?.,\n]', '', text)
    text = re.sub(r'\n', ' ', text) # 줄바꿈을 공백으로
    text = re.sub(r'\s+', ' ', text).strip()
    return text

combined_df['clean_text'] = combined_df['conversation'].apply(custom_preprocessing)
test_df['clean_text'] = test_df['conversation'].apply(custom_preprocessing)

# ==========================================
# 3. 라벨 인코딩 (오류 해결 핵심)
# ==========================================
label_map = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

# [핵심] class 컬럼을 문자열로 강제 변환 후 매핑 (숫자형 0 등이 섞여 있을 경우 대비)
combined_df['class'] = combined_df['class'].astype(str).str.strip()
combined_df['label'] = combined_df['class'].map(label_map)

# 매핑 실패(NaN) 확인 및 제거
if combined_df['label'].isnull().sum() > 0:
    print(f"⚠️ 경고: {combined_df['label'].isnull().sum()}개의 라벨 매핑 실패. 해당 데이터를 제외합니다.")
    print("매핑 실패한 클래스 예시:", combined_df[combined_df['label'].isnull()]['class'].unique())
    combined_df = combined_df.dropna(subset=['label'])

# 라벨을 정수형(int)으로 변환
combined_df['label'] = combined_df['label'].astype(int)

print("데이터 준비 완료. 라벨 분포:")
print(combined_df['label'].value_counts())

# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(
    combined_df['clean_text'],
    combined_df['label'],
    test_size=0.2,
    random_state=SEED,
    stratify=combined_df['label']
)

# ==========================================
# 4. 데이터셋 및 모델 정의
# ==========================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class ThreatDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        item_dict = {key: val.flatten() for key, val in encoding.items()}

        # [중요] 라벨을 반드시 LongTensor(정수)로 변환
        if self.labels is not None:
            item_dict['labels'] = torch.tensor(self.labels[item], dtype=torch.long)

        return item_dict

train_dataset = ThreatDataset(X_train.tolist(), y_train.tolist(), tokenizer, MAX_LEN)
val_dataset = ThreatDataset(X_val.tolist(), y_val.tolist(), tokenizer, MAX_LEN)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5)
model.to(device)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # [안전장치] 혹시라도 타입이 섞여있을까봐 int로 변환
    labels = labels.astype(int)
    preds = preds.astype(int)

    f1 = f1_score(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1_macro': f1}

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    eval_strategy="epoch",  # evaluation_strategy -> eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    save_total_limit=1,
    fp16=True if torch.cuda.is_available() else False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# ==========================================
# 5. 학습 및 추론
# ==========================================
print("Training Start...")
trainer.train()

print("Inference Start...")
test_texts = test_df['clean_text'].tolist()
test_dataset = ThreatDataset(test_texts, labels=None, tokenizer=tokenizer, max_len=MAX_LEN)

predictions = trainer.predict(test_dataset)
pred_ids = np.argmax(predictions.predictions, axis=1)

# 복원
id_to_label = {v: k for k, v in label_map.items()}
pred_strings = [id_to_label[pid] for pid in pred_ids]

# 제출 파일 생성
try:
    submission = pd.read_csv('/content/drive/MyDrive/DLThon/data/submission.csv')
    submission['class'] = pred_strings
    submission.to_csv('/content/drive/MyDrive/DLThon/result/final_result.csv', index=False, encoding='utf-8-sig')
    print("성공! 'final_result.csv' 생성 완료.")
except Exception as e:
    print(f"제출 파일 생성 중 오류 발생: {e}")
    # 대체 방법
    sub_df = pd.DataFrame({'idx': test_df['idx'], 'class': pred_strings})
    sub_df.to_csv('/content/drive/MyDrive/DLThon/result/final_result_backup.csv', index=False, encoding='utf-8-sig')
    print("'final_result_backup.csv'로 저장되었습니다.")

Using device: cuda
Loading data...
데이터 준비 완료. 라벨 분포:
label
3    1094
1     981
2     979
0     896
4     800
Name: count, dtype: int64


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: klue/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
classifier.dense.bias           | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.weight         | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Training Start...


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,No log,0.386237,0.860000,0.867926
2,No log,0.328789,0.908421,0.912417
3,0.404877,0.332617,0.908421,0.912114


...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.3.output.Laye

Inference Start...


성공! 'final_result.csv' 생성 완료.


In [7]:
import pandas as pd
import numpy as np
import torch
import random
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# 1. 설정 및 시드 고정
SEED = 42
MODEL_NAME = "klue/roberta-base"
MAX_LEN = 128  # 메모리 부족 시 128, 여유 있으면 256
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ==========================================
# 2. 데이터 로드 및 전처리 (안전 버전)
# ==========================================
print("Loading data...")
try:
    train_df = pd.read_csv('/content/drive/MyDrive/DLThon/data/train.csv')
    normal_df = pd.read_csv('/content/drive/MyDrive/DLThon/normal_conv/normal_conversation.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/DLThon/data/test.csv')
except Exception as e:
    print(f"파일 로드 중 오류 발생: {e}")
    # 코랩 등에서 파일 경로 확인 필요

# 데이터 병합
combined_df = pd.concat([train_df, normal_df], ignore_index=True)

# [중요] 전처리 함수
def custom_preprocessing(text):
    if not isinstance(text, str): return ""
    text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ]){3,}', r'\1\1', text)
    text = re.sub(r'([!?.]{2,})', r'\1', text)
    text = re.sub(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 !?.,\n]', '', text)
    text = re.sub(r'\n', ' ', text) # 줄바꿈을 공백으로
    text = re.sub(r'\s+', ' ', text).strip()
    return text

combined_df['clean_text'] = combined_df['conversation'].apply(custom_preprocessing)
test_df['clean_text'] = test_df['conversation'].apply(custom_preprocessing)

# ==========================================
# 3. 라벨 인코딩 (오류 해결 핵심)
# ==========================================
label_map = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

# [핵심] class 컬럼을 문자열로 강제 변환 후 매핑 (숫자형 0 등이 섞여 있을 경우 대비)
combined_df['class'] = combined_df['class'].astype(str).str.strip()
combined_df['label'] = combined_df['class'].map(label_map)

# 매핑 실패(NaN) 확인 및 제거
if combined_df['label'].isnull().sum() > 0:
    print(f"⚠️ 경고: {combined_df['label'].isnull().sum()}개의 라벨 매핑 실패. 해당 데이터를 제외합니다.")
    print("매핑 실패한 클래스 예시:", combined_df[combined_df['label'].isnull()]['class'].unique())
    combined_df = combined_df.dropna(subset=['label'])

# 라벨을 정수형(int)으로 변환
combined_df['label'] = combined_df['label'].astype(int)

print("데이터 준비 완료. 라벨 분포:")
print(combined_df['label'].value_counts())

# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(
    combined_df['clean_text'],
    combined_df['label'],
    test_size=0.2,
    random_state=SEED,
    stratify=combined_df['label']
)

# ==========================================
# 4. 데이터셋 및 모델 정의
# ==========================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class ThreatDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        item_dict = {key: val.flatten() for key, val in encoding.items()}

        # [중요] 라벨을 반드시 LongTensor(정수)로 변환
        if self.labels is not None:
            item_dict['labels'] = torch.tensor(self.labels[item], dtype=torch.long)

        return item_dict

train_dataset = ThreatDataset(X_train.tolist(), y_train.tolist(), tokenizer, MAX_LEN)
val_dataset = ThreatDataset(X_val.tolist(), y_val.tolist(), tokenizer, MAX_LEN)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5)
model.to(device)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # [안전장치] 혹시라도 타입이 섞여있을까봐 int로 변환
    labels = labels.astype(int)
    preds = preds.astype(int)

    f1 = f1_score(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1_macro': f1}

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    eval_strategy="epoch",  # evaluation_strategy -> eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    save_total_limit=1,
    fp16=True if torch.cuda.is_available() else False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# ==========================================
# 5. 학습 및 추론
# ==========================================
print("Training Start...")
trainer.train()

print("Inference Start...")
test_texts = test_df['clean_text'].tolist()
test_dataset = ThreatDataset(test_texts, labels=None, tokenizer=tokenizer, max_len=MAX_LEN)

predictions = trainer.predict(test_dataset)
pred_ids = np.argmax(predictions.predictions, axis=1)

# 복원
id_to_label = {v: k for k, v in label_map.items()}
pred_strings = [id_to_label[pid] for pid in pred_ids]

# 제출 파일 생성
try:
    submission = pd.read_csv('/content/drive/MyDrive/DLThon/data/submission.csv')
    submission['class'] = pred_strings
    submission.to_csv('/content/drive/MyDrive/DLThon/result/final_result.csv', index=False, encoding='utf-8-sig')
    print("성공! 'final_result.csv' 생성 완료.")
except Exception as e:
    print(f"제출 파일 생성 중 오류 발생: {e}")
    # 대체 방법
    sub_df = pd.DataFrame({'idx': test_df['idx'], 'class': pred_strings})
    sub_df.to_csv('/content/drive/MyDrive/DLThon/result/final_result_backup.csv', index=False, encoding='utf-8-sig')
    print("'final_result_backup.csv'로 저장되었습니다.")

Using device: cuda
Loading data...
데이터 준비 완료. 라벨 분포:
label
3    1094
1     981
2     979
0     896
4     800
Name: count, dtype: int64


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: klue/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
classifier.dense.bias           | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.weight         | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Training Start...


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,No log,0.386237,0.860000,0.867926
2,No log,0.328789,0.908421,0.912417
3,0.404877,0.332617,0.908421,0.912114


...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.3.output.Laye

Inference Start...


성공! 'final_result.csv' 생성 완료.


In [8]:
import pandas as pd

# 1. 기존 결과 파일 읽기 (방금 만든 파일)
try:
    df = pd.read_csv('/content/drive/MyDrive/DLThon/result/final_result.csv')
    print("기존 결과 파일 로드 성공")
except:
    # 만약 파일이 없다면, 변수(submission)가 메모리에 있다고 가정
    df = submission.copy()

# 2. 문자열 -> 숫자 매핑 (학습 때 사용한 것과 동일해야 함)
label_to_num = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

# 3. 변환 수행
# 만약 이미 숫자로 되어 있다면 건너뜀
if df['class'].dtype == 'object' or df['class'].dtype == 'O':
    df['class'] = df['class'].map(label_to_num)
    print("문자열 -> 숫자로 변환 완료")
else:
    print("이미 숫자로 되어 있습니다.")

# 4. (중요) 혹시 NaN(변환 안 된 것)이 있는지 확인
if df['class'].isnull().sum() > 0:
    print(f"경고: {df['class'].isnull().sum()}개의 항목이 변환되지 않았습니다. 매핑 딕셔너리를 확인하세요.")
    # NaN은 4(일반 대화)나 0으로 채우기
    df['class'] = df['class'].fillna(4).astype(int)

# 5. 저장
df.to_csv('/content/drive/MyDrive/DLThon/result/final_result_numeric.csv', index=False)
print("저장 완료: final_result_numeric.csv")
print(df.head())

기존 결과 파일 로드 성공
문자열 -> 숫자로 변환 완료
저장 완료: final_result_numeric.csv
     idx  class
0  t_000      1
1  t_001      2
2  t_002      2
3  t_003      3
4  t_004      3
